In [2]:
import cv2
import numpy as np
import mediapipe as mp
import os
from datetime import datetime

# Configuration
SAVE_FOLDER = "asl_dataset_right_hand"  # Folder to save images
os.makedirs(SAVE_FOLDER, exist_ok=True)  # Create folder if it doesn't exist

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

def draw_hand_landmarks_diagram(hand_landmarks, image_size=128, padding_factor=0.3):
    """Create a clean hand landmark diagram with white lines on black background"""
    landmarks = np.array([[lm.x, lm.y] for lm in hand_landmarks.landmark])
    
    # Calculate bounds with padding
    x_vals = landmarks[:, 0]
    y_vals = landmarks[:, 1]
    width = np.max(x_vals) - np.min(x_vals)
    height = np.max(y_vals) - np.min(y_vals)
    
    pad_x = width * padding_factor
    pad_y = height * padding_factor
    
    min_x = np.min(x_vals) - pad_x
    max_x = np.max(x_vals) + pad_x
    min_y = np.min(y_vals) - pad_y
    max_y = np.max(y_vals) + pad_y
    
    # Normalize to image size
    hand_width = max_x - min_x
    hand_height = max_y - min_y
    scale = image_size / max(hand_width, hand_height) * 0.9
    
    landmarks[:, 0] = (landmarks[:, 0] - min_x) * scale
    landmarks[:, 1] = (landmarks[:, 1] - min_y) * scale
    
    # Center in image
    offset_x = (image_size - (hand_width * scale)) / 2
    offset_y = (image_size - (hand_height * scale)) / 2
    landmarks[:, 0] += offset_x
    landmarks[:, 1] += offset_y
    
    # Create black canvas
    diagram = np.zeros((image_size, image_size), dtype=np.uint8)
    points = landmarks.astype(np.int32)
    
    # Draw connections (white lines)
    for connection in mp_hands.HAND_CONNECTIONS:
        start = tuple(points[connection[0]])
        end = tuple(points[connection[1]])
        cv2.line(diagram, start, end, 255, 1)
    
    # Draw landmarks (white points)
    for point in points:
        cv2.circle(diagram, tuple(point), 1, 255, -1)
    
    return diagram

# Start webcam
cap = cv2.VideoCapture(0)
save_counter = 0

print("ASL Dataset Creation Tool")
print("Press:")
print("- 's' to save current hand diagram")
print("- '1-9' to set current class label")
print("- ESC to exit")

current_label = "A"  # Default label

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    # Display current label
    cv2.putText(frame, f"Current label: {current_label}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Saved: {save_counter}", (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Create the diagram
            diagram = draw_hand_landmarks_diagram(hand_landmarks)
            
            # Show preview
            cv2.imshow("Diagram Preview", diagram)
            
            # Draw landmarks on webcam feed
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            break

    cv2.imshow("ASL Dataset Creator", frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # ESC key
        break
    elif key == ord('a') and results.multi_hand_landmarks:
        # Save the diagram
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = f"{SAVE_FOLDER}/{current_label}_{timestamp}.png"
        cv2.imwrite(filename, diagram)
        save_counter += 1
        print(f"Saved {filename}")
    elif ord('1') <= key <= ord('9'):
        # Change label (1=A, 2=B, ..., 9=I)
        current_label = chr(ord('A') + (key - ord('1')))
    elif key == ord('0'):
        # Set label to J (0 key)
        current_label = 'J'
    elif ord('a') <= key <= ord('z'):
        # Change label to pressed letter
        current_label = chr(key).upper()

cap.release()
cv2.destroyAllWindows()

ASL Dataset Creation Tool
Press:
- 's' to save current hand diagram
- '1-9' to set current class label
- ESC to exit
Saved asl_dataset_right_hand/A_20250607_122308_923046.png
Saved asl_dataset_right_hand/A_20250607_122309_243606.png
Saved asl_dataset_right_hand/A_20250607_122309_515281.png
Saved asl_dataset_right_hand/A_20250607_122309_700453.png
Saved asl_dataset_right_hand/A_20250607_122309_946120.png
Saved asl_dataset_right_hand/A_20250607_122310_138727.png
Saved asl_dataset_right_hand/A_20250607_122310_395125.png
Saved asl_dataset_right_hand/A_20250607_122310_620593.png
Saved asl_dataset_right_hand/A_20250607_122310_859235.png
Saved asl_dataset_right_hand/A_20250607_122311_083517.png
Saved asl_dataset_right_hand/A_20250607_122311_339417.png
Saved asl_dataset_right_hand/A_20250607_122311_547858.png
Saved asl_dataset_right_hand/A_20250607_122311_724618.png
Saved asl_dataset_right_hand/A_20250607_122311_906205.png
Saved asl_dataset_right_hand/A_20250607_122312_172410.png
Saved asl_dat